<a href="https://colab.research.google.com/github/ccwilliamsut/machine_learning/blob/master/MLAB_06_Grid_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build a Grid Search Model (w/ Cross-Validation)
In the previous section (Build a Gradient Boosting Model), we learned how to build multiple decision trees that work sequentially to create a more effective model.

One of the **problems** associated with this method, however, is the sheer **number of interconnected hyperparameters**. Because each of the hyperparameters play off of one another, **determining the optimal settings can prove difficult and time-consuming**.

There are tools to help with finding optimal settings in a Gradient Boosting model, however. Using **GridSearchCV**, one can apply **multiple hyperparameter settings at once** to a Gradient Boosting model and methodically process all possible combinations automatically. At the end of the run, GridSearchCV will produce a set of **optimal hyperparameters for a Gradient Boosting model** (given the possible inputs).

## A. Setup the Environment

In [0]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> SETUP ENVIRONMENT <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# Import libraries
import pandas as pd
import seaborn as sns; sns.set()
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from IPython.display import display
from sklearn import preprocessing
from sklearn.model_selection import train_test_split 
from sklearn import ensemble
from scipy import stats
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import learning_curve,GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import learning_curve
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

url = 'https://raw.githubusercontent.com/ccwilliamsut/machine_learning/master/absolute_beginners/data_files/modified/CaliforniaHousingDataModified.csv'

df = pd.read_csv(url)
#df = pd.read_csv(~/Downloads/CaliforniaHousingDataModified.csv)


# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> SETUP DATA <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

# --------------------------------------------- RENAME FEATURES ---------------------------------------------
df.rename(columns = {'lattitude':'latitude', 't_rooms':'total_rooms'}, inplace=True)


# --------------------------------------------- ONE-HOT ENCODING ---------------------------------------------
df['ocean_proximity'] = pd.Categorical(df['ocean_proximity'])
df_dummies = pd.get_dummies(df['ocean_proximity'], drop_first = True)
df.drop(['ocean_proximity'], axis = 1, inplace = True)
df = pd.concat([df, df_dummies], axis=1)


# --------------------------------------------- DROP UNWANTED FEATURES ---------------------------------------------
df.drop(['proximity_to_store'], axis = 1, inplace = True)


# ------------------------------------- FIX MISSING DATA -------------------------------------
tb_med = df['total_bedrooms'].median(axis=0)
df['total_bedrooms'] = df['total_bedrooms'].fillna(value = tb_med)
df['total_bedrooms'] = df['total_bedrooms'].astype(int)
df.name = 'df'

# ------------------------------------- Z-SCORE -------------------------------------
z = np.abs(stats.zscore(df))
dfz = df[(z < 3).all(axis = 1)]
dfz.name = 'dfz'

# ------------------------------------- INTERQUARTILE RANGE -------------------------------------
q1 = df.quantile(0.25)
q3 = df.quantile(0.75)
iqr = q3 - q1
lower = q1 - (1.5 * iqr)
upper = q3 + (1.5 * iqr)
dfi = df[~((df < lower) | (df > upper)).any(axis = 1)]
dfi.name = 'dfi'
#dfi = dfi.drop(['NEAR BAY', 'NEAR OCEAN'], axis = 1)  # After applying IQR, the following features are now empty and can be dropped

# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> FUNCTIONS <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
def make_heatmap(df = 'df'):
  corr = df.corr()
  plt.figure(figsize=(15 ,9))
  sns.heatmap(corr, annot=True, vmin = -1, vmax = 1, center = 0, fmt = '.1g', cmap = 'coolwarm')
  plt.show()
  plt.close()


def make_heading(heading = 'heading'):
  print('\n\n{0}:\n'.format(heading), '-' * 30)


def drop_features(df_in):
  df_in = df_in.drop([#'median_house_value',
                      'population',
                      'households',
                      'longitude',
                      'latitude',
                      #'housing_median_age',
                      'total_rooms',
                      'total_bedrooms',
                      #'median_income',
                      'INLAND',
                      'ISLAND',
                      #'NEAR BAY',
                      #'NEAR OCEAN'
                      ],
                      axis = 1
                      )
  return features_dfx


def plot_test_predictions(y_test, y_pred):
  make_heading('Prediction Performance')  # Make a heading to separate output
  fig, ax = plt.subplots()
  ax = plt.subplot()
  ax.scatter(y_test, y_pred, edgecolors=(0, 0, 0), alpha=0.5)
  ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'g--', lw=4)
  ax.set_xlabel('Actual')
  ax.set_ylabel('Predicted')
  ax.set_title("Ground Truth vs Predicted")
  plt.show()
  plt.close()


def plot_feature_importance(feature_importance):
  # Make importances relative to max importance
  feature_importance = 100.0 * (feature_importance / feature_importance.max())
  sorted_idx = np.argsort(feature_importance)
  pos = np.arange(sorted_idx.shape[0]) + .5
  make_heading('Feature Importance')  # Make a heading to separate output
  plt.subplot(1, 2, 2)
  plt.barh(pos, feature_importance[sorted_idx], align='center')
  plt.yticks(pos, features_dfx)
  plt.xlabel('Relative Importance')
  plt.title('Variable Importance')
  plt.show()
  plt.close()


def plot_deviance():
  # Plot the training/testing accuracy
  n_est = gs_cv.best_estimator_.n_estimators_
  y_pred = gs_cv.best_estimator_.predict(X_test)
  y_staged_predicted_score = gs_cv.best_estimator_.staged_predict(X_test)
  model_test_score = gs_cv.best_estimator_.score(X_test, y_test)  # Accuracy of our model on test data
  model_train_score = gs_cv.best_estimator_.score(X_train, y_train)  # Accuracy of our model on training data
  

  # Create an empty array of zeros based on the value in 'n_estimators' key
  test_score = np.zeros(shape = (n_est,),
                        dtype=np.float64
                        )

  # Compute test set deviance (loss) at each stage and place it into the array
  #  based on the predicted value (y_pred) against the actual value (y_test)
  for i, y_pred in enumerate(y_staged_predicted_score):
      test_score[i] = gs_cv.best_estimator_.loss_(y_test, y_pred)
  
  make_heading('Deviance Over Time')  # Make a heading to separate output
  plt.figure(figsize=(15, 7))
  plt.subplot(1, 2, 1)
  plt.title('Deviance')
  plt.plot(np.arange(n_est) + 1, 
           gs_cv.best_estimator_.train_score_,
           'b-',
           label='Training Set Deviance'
           )
  plt.plot(np.arange(n_est) + 1,
           test_score,
           'r-',
           label='Test Set Deviance'
           )
  plt.legend(loc='upper right')
  plt.xlabel('Boosting Iterations')
  plt.ylabel('Deviance')
  plt.show()
  plt.close
  

def show_metrics(X_train, X_test, y_train, y_test, y_pred, cv_best_score_mean):
  # Display the shape of each array:
  #make_heading('The shape of each of our arrays after splitting into training and testing sets for dataframe \"{0}\"'.format(dfx.name))
  #print('X_train shape: ', X_train.shape)
  #print('X_test shape:  ', X_test.shape)
  #print('y_train shape: ', y_train.shape)
  #print('y_test shape:  ', y_test.shape)
  #print('y_pred shape:  ', y_pred.shape)

  # Display training / testing set metrics
  make_heading('\n\nAccuracy and Error for training/testing on the {0} model for dataframe \"{1}\"'.format(model_name, dfx.name))
  print("Training Accuracy (score)                 (X_train, y_train):  {:.2f}".format(model_train_score))
  print("Test Accuracy (score)                     (X_test, y_test):    {:.2f}".format(model_test_score))
  print("Predictive Accuracy (R^2 score)           (y_test, y_pred):    {:.2f}".format(predictive_accuracy))
  print("Explained Variance (1 is best) (loss)     (y_test, y_pred):    {:.2f}".format(ev))
  print("Mean Absolute Error on Test Set (loss)    (y_test, y_pred):    {:.2f}".format(mean_ae))
  print("Median Absolute Error on Test Set (loss)  (y_test, y_pred):    {:.2f}".format(median_ae))
  print("RMSE on Test Set (loss)                   (y_test, y_pred):    {:.2f}".format(rmse))
  print("Best Cross-Validated Score (mean)         gs_cv.best_score_:   {:.2f}".format(cv_best_score_mean))

## B. Choose a dataset to use
- Three datasets have been created:
  1. **df** -- the **original dataset** ('NaN' values have been replaced with median, so there are no null values)
  2. **dfz** -- a dataset that has used the **z-score method** for removing outlier data
  3. **dfi** -- a dataset that has used the **IQR method** for removing outlier data

```

```

**NOTE:** When choosing your dataset, you will need to change two commands to reflect the dataset that you want to use. In the following example, the 

**Original code:** --------------------------------------->  **New code** (For example, to change **from** 'dfz' **to** 'dfi' dataset):
```
dfx = dfz.copy()      -->   dfx = dfi.copy()
dfx.name = dfz.name   -->   dfx.name = dfi.name
```



```
  
  

```

In [0]:
# *************************** DETERMINE DATA FOR USE ***************************
# Choose which dataframe you would like to use (dfi: IQR, dfz: z-score, df: original (with replaced NaN values))
dfx = df.copy()
dfx.name = df.name

# Show a heatmap for the given dataframe you want to use
make_heatmap(dfx)

## C. Step by Step: Building a Gradient Boosting Model

### From *Machine Learning For Absolute Beginners* by Oliver Theobald
```

model = ensemble.GradientBoostingRegressor(
                                           n_estimators = 150, 
                                           learning_rate = 0.1, 
                                           max_depth = 30, 
                                           min_samples_split = 4, 
                                           min_samples_leaf = 6, 
                                           max_features = 0.6, 
                                           loss = 'huber'
                                          )
```

The first line is the algorithm itself (gradient boosting) and comprises just one line of code. The code below dictates the hyperparameters for this algorithm. 
- **n_estimators** represents how many decision trees to be used. Remember that a high number of trees generally improves accuracy (up to a certain point) but will extend the model’s processing time. Above, I have selected 150 decision trees as an initial starting point. 
- **learning_rate** controls the rate at which additional decision trees influence the overall prediction. This effectively shrinks the contribution of each tree by the set learning_rate. Inserting a low rate here, such as 0.1, should help to improve accuracy. 
- **max_depth** defines the maximum number of layers (depth) for each decision tree. If “None” is selected, then nodes expand until all leaves are pure or until all leaves contain less than min_samples_leaf. Here, I have chosen a high maximum number of layers (30), which will have a dramatic effect on the final result, as we’ll soon see. 
- [**min_samples_split**](https://stackoverflow.com/questions/46480457/difference-between-min-samples-split-and-min-samples-leaf-in-sklearn-decisiontre) defines the minimum number of samples required to execute a new binary split. For example, min_samples_split = 10 means there must be ten available samples in order to create a new branch.
- [**min_samples_leaf**](**min_samples_split**) represents the minimum number of samples that must appear in each child node (leaf) before a new branch can be implemented. This helps to mitigate the impact of outliers and anomalies in the form of a low number of samples found in one leaf as a result of a binary split. For example, min_samples_leaf = 4 requires there to be at least four available samples within each leaf for a new branch to be created. 
- **max_features** is the total number of features presented to the model when determining the best split.
- **loss** calculates the model's error rate. For this exercise, we are using huber which protects against outliers and anomalies. Alternative error rate options include ls (least squares regression), lad (least absolute deviations), and quantile (quantile regression). Huber is actually a combination of least squares regression and least absolute deviations.


  >Theobald, Oliver. Machine Learning For Absolute Beginners: A Plain English Introduction (Second Edition) (Machine Learning For Beginners Book 1) (pp. 139-141). Scatterplot Press. Kindle Edition.

> Websites Referenced:
- [Scikit-Learn Documentation: Ensemble Gradient Boosting Visualization](https://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_regression.html#sphx-glr-auto-examples-ensemble-plot-gradient-boosting-regression-py)
- [Sharp Sight Labs: Numpy Zeros Tutorial](https://www.sharpsightlabs.com/blog/numpy-zeros-python/)
- [Towards Data Science: Beginner's Guide to Cross-Validation](https://towardsdatascience.com/cross-validation-a-beginners-guide-5b8ca04962cd)
- [Scikit-Learn Documentation: Decision Tree Regressor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)
- [Scikit-Learn Documentation: Tuning the hyper-parameters of an estimator (Grid Search)](https://scikit-learn.org/stable/modules/grid_search.html)
- [GDCoder.com: Decision Tree Regressor explained in depth](https://gdcoder.com/decision-tree-regressor-explained-in-depth/)
- [link text](https://)

In [0]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> STEP BY STEP DECISION TREE CONSTRUCTION <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

# ---------------------------- Identify Target Variable ----------------------------
# Create a copy of the dataframe with only the desired features (dropping the target feature)
features_dfx = dfx.copy()
features_dfx = dfx.drop(['median_house_value'], axis = 1)  # We *MUST* drop this b/c it is the target

# Determine additional features to keep/drop (# means that we want to keep that variable)
features_dfx = features_dfx.drop([#'longitude',
                                  #'latitude',
                                  #'housing_median_age',
                                  #'total_rooms',
                                  #'total_bedrooms',
                                  #'population',
                                  #'households',
                                  #'median_income',
                                  #'median_house_value',
                                  #'INLAND',
                                  'ISLAND',
                                  'NEAR BAY',
                                  'NEAR OCEAN'
                                  ],
                                 axis = 1
                                 )


# ---------------------------- SPLIT THE DATASET ----------------------------
X = features_dfx.values                 # Define the independent variable values to be used
y = dfx['median_house_value'].values    # Define the dependent variable values to be used
rs = 20                                 # Define the random state variable (ensuring continuity between runs)
test_size = 0.2                         # Define the percentage of data to set aside for testing (usually b/w 0.2 - 0.3)

# Split the dataset into training and testing arrays
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = test_size,
                                                    shuffle = True,
                                                    random_state = rs
                                                    )


# ---------------------------- CREATE THE MODEL  ----------------------------
# -- Set the hyperparameters that we want to use in the model
"""
param_grid = {'n_estimators': [50, 100, 200],
              'max_depth': [3, 5],
              'min_samples_split': [2, 5, 8],
              'min_samples_leaf': [2, 4],
              'learning_rate': [0.1, 1.0],
              'max_features': [0.6, 0.8],
              'loss': ['huber']
              }
"""
"""
param_grid = {'n_estimators': [400],
              'max_depth': [7],
              'min_samples_split': [72],
              'min_samples_leaf': [20],
              'learning_rate': [0.1],
              'max_features': [0.8],
              'loss': ['huber']
              }

"""

param_grid = {'n_estimators': [500],
              'max_depth': [5],
              'min_samples_split': [16],
              'min_samples_leaf': [10],
              'learning_rate': [0.1],
              'max_features': [0.8],
              'loss': ['huber']
              }



# Define the desired algorithm and congigure the hyperparameters (supplied with '**params' arguement)
base_model = ensemble.GradientBoostingRegressor()
base_model_name = type(base_model).__name__                 # Get the name of the model (for use in our display functions)


# ---------------------------- CREATE THE GRID SEARCH MODEL ----------------------------
gs_cv = GridSearchCV(estimator = base_model,
                     param_grid = param_grid,
                     n_jobs = -1, 
                     cv = 3,
                     return_train_score = True,
                     verbose = 10,
                     refit = True
                     )


# ---------------------------- FIT / TRAIN THE MODEL ----------------------------
gs_cv.fit(X_train, y_train)                                 # Train the model using our training sets (X_train, y_train)
y_pred = gs_cv.predict(X_test)                              # Make predictions based upon our trained model

# ---------------------------- GATHER METRICS ----------------------------

model_name = type(gs_cv).__name__                           # Get the name of our model (for use in displaying stats)
model_test_score = gs_cv.score(X_test, y_test)              # Accuracy of our model on test data
model_train_score = gs_cv.score(X_train, y_train)           # Accuracy of our model on training data
mean_ae = metrics.mean_absolute_error(y_test, y_pred)       # Mean absolute error (find mae on the test set (to see how well the trained model performs on new data(y_test against y_pred))
median_ae = metrics.median_absolute_error(y_test, y_pred)   # Median absolute error
rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))  # Root mean squared error
ev = metrics.explained_variance_score(y_test, y_pred)       # Explained Variance Score (y_test, y_pred): Measures the proportion to which a mathematical model accounts for the variation (dispersion) of a given data set
predictive_accuracy = metrics.r2_score(y_test, y_pred)      # Determine the predictive accuracy of the model by getting the R^2 score (how well future samples are likely to be predicted by the model)
cv_best_score_mean = gs_cv.best_score_                      # Get the mean cross-validated score of the best_esimator
mae_train = metrics.mean_absolute_error(y_train, gs_cv.predict(X_train))  
mae_test = metrics.mean_absolute_error(y_test, gs_cv.predict(X_test))  


# ---------------------------- ANALYSIS / VISUALIZATION ----------------------------
make_heading('Testing and Training Mean Absolute Error Data')
print ("Training Set Mean Absolute Error: %.2f" % mae_train)  
print ("Test Set Mean Absolute Error: %.2f" % mae_test)
make_heading('Hyperparameters Explored in GridSearch')
print(pd.Series(param_grid))
make_heading('Best Hyperparameters')
print(pd.Series(gs_cv.best_params_))
show_metrics(X_train, X_test, y_train, y_test, y_pred, cv_best_score_mean)  # Dispay the scores and loss for the model
plot_test_predictions(y_test, y_pred)                       # Create a scatterplot of real values against predicted ones for the test set
feature_importance = gs_cv.best_estimator_.feature_importances_  # Gather feature importance values
plot_feature_importance(feature_importance)                 # Plot feature importance
plot_deviance()                                             # Plot training / testing accuracy

# Save the trained model for future use
joblib.dump(gs_cv.best_estimator_, 'ca_housing_gb_model.pkl')